<a href="https://colab.research.google.com/github/britoadriana/Mensagens_164/blob/main/RAG_relatorios_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Usando OpenAI Embeddings em RAG com MongoDB




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Step 1: Libraries Installation


Below are brief explanations of the tools and libraries utilised within the implementation code:
* **datasets**: This library is part of the Hugging Face ecosystem. By installing 'datasets', we gain access to a number of pre-processed and ready-to-use datasets, which are essential for training and fine-tuning machine learning models or benchmarking their performance.

* **pandas**: A data science library that provides robust data structures and methods for data manipulation, processing and analysis.

* **openai**: This is the official Python client library for accessing OpenAI's suite of AI models and tools, including GPT and embedding models.italicised text

* **pymongo**: PyMongo is a Python toolkit for MongoDB. It enables interactions with a MongoDB database.

In [ ]:
!pip install datasets pandas openai pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.2/677.2 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00


## Step 2: Data Loading


Usei uma tabela com dados de relatórios do IPT do ano de 2023.


In [ ]:
# 1. Load Dataset

import pandas as pd
dataset_df = pd.read_excel('/content/drive/MyDrive/HF/data.xls')
dataset_df.head(5)

,Tipo do documento,Nº do documento,Ano de emissão,Área,Laboratorio,CRD,Cliente,CNPJ/CPF/CRD do cliente,Interessado,Titulo,Nº do projeto,Projeto de capacitação,Cancelado e substituido por,Formato do documento,Data de emissão,Descritores,Resumo,Gerente,Equipe
0,Relatório Técnico,166547,2023,TRM,LV,DH14,GASCAT INDUSTRIA E COMERCIO LTDA,00965449000194,NaN,AVALIAÇÃO DE UM SISTEMA DE MEDIÇÃO DE VOLUME T...,810103C,810103C,NaN,Digital,24/03/2023 18:36:16,ROTATIVO;TEMPERATURA;PRESSÃO,Avaliação de um sistema de medição de volume t...,RUBENS SILVA TELLES,Autor: Jose Eduardo Prata;Equipe: Fernando Hen...
1,Relatório Técnico,166548,2023,TRM,LV,DH14,GASCAT INDUSTRIA E COMERCIO LTDA,00965449000194,NaN,AVALIAÇÃO DE UM SISTEMA DE MEDIÇÃO DE VOLUME T...,810103C,810103C,NaN,Digital,24/03/2023 18:37:29,ROTATIVO;TEMPERATURA;PRESSÃO,Avaliação de um sistema de medição de volume t...,RUBENS SILVA TELLES,Autor: Jose Eduardo Prata;Equipe: Fernando Hen...
2,Relatório Técnico,166549,2023,TRM,LV,DH14,GASCAT INDUSTRIA E COMERCIO LTDA,00965449000194,NaN,AVALIAÇÃO DE UM SISTEMA DE MEDIÇÃO DE VOLUME T...,810103C,810103C,NaN,Digital,24/03/2023 18:38:37,TURBINA;TEMPERATURA;PRESSÃO,Avaliação de um sistema de medição de volume t...,RUBENS SILVA TELLES,Autor: Jose Eduardo Prata;Equipe: Fernando Hen...
3,Relatório Técnico,166550,2023,TRM,LV,DH14,GASCAT INDUSTRIA E COMERCIO LTDA,00965449000194,NaN,AVALIAÇÃO DE UM SISTEMA DE MEDIÇÃO DE VOLUME T...,810103C,810103C,NaN,Digital,17/04/2023 17:26:32,TURBINA;TEMPERATURA;PRESSÃO,Avaliação de um sistema de medição de volume t...,RUBENS SILVA TELLES,Autor: Jose Eduardo Prata;Equipe: Fernando Hen...
4,Relatório Técnico,166672,2023,MA,LCPE,DO13,FUNDO NACIONAL DE DESENVOLVIMENTO DA EDUCACAO,00378257000181,NaN,ANÁLISE DA QUALIDADE FÍSICA DE LIVROS ESCOLARE...,500209B,não,NaN,Digital,22/09/2023 12:51:03,LIVRO ESCOLAR; FNDE; PNLD.,Este Relatório Técnico apresenta os resultados...,PATRÍCIA KAJI YASUMURA,Autor: PATRÍCIA KAJI YASUMURA;Equipe: PATRICIA...


## Step 3: Data Cleaning and Preparation

The next step cleans the data and prepares it for the next stage, which creates a new embedding data point using the new OpenAI embedding model.


In [ ]:
print("Columns:", dataset_df.columns)
print("\nNumber of rows and columns:", dataset_df.shape)
print("\nBasic Statistics for numerical data:")
print(dataset_df.describe())
print("\nNumber of missing values in each column:")
print(dataset_df.isnull().sum())

Columns: Index(['Tipo do documento', 'Nº do documento', 'Ano de emissão', 'Área',
       'Laboratorio', 'CRD', 'Cliente', 'CNPJ/CPF/CRD do cliente',
       'Interessado', 'Titulo', 'Nº do projeto', 'Projeto de capacitação',
       'Cancelado e substituido por', 'Formato do documento',
       'Data de emissão', 'Descritores', 'Resumo', 'Gerente', 'Equipe'],
      dtype='object')

Number of rows and columns: (2494, 19)

Basic Statistics for numerical data:
       Nº do documento  Ano de emissão
count      2494.000000          2494.0
mean     169244.031275          2023.0
std         994.257491             0.0
min      166547.000000          2023.0
25%      168520.250000          2023.0
50%      169204.500000          2023.0
75%      169916.750000          2023.0
max      171212.000000          2023.0

Number of missing values in each column:
Tipo do documento                 0
Nº do documento                   0
Ano de emissão                    0
Área                              0
Labo

## Step 4: Create embeddings with OpenAI

This stage focuses on generating new embeddings using OpenAI's advanced model.
This demonstration utilises a Google Colab Notebook, where environment variables are configured explicitly within the notebook's Secret section and accessed using the user data module. In a production environment, the environment variables that store secret keys are usually stored in a '.env' file or equivalent.

An [OpenAI API](https://help.openai.com/en/articles/4936850-where-do-i-find-my-api-key) key is required to ensure the successful completion of this step. More details on OpenAI's embedding models can be found on the official [site](https://platform.openai.com/docs/guides/embeddings).


In [ ]:
import openai
from google.colab import userdata

caminho_para_chave = '/content/drive/MyDrive/HF/chave_adri_openai.txt'

with open(caminho_para_chave, 'r') as arquivo:
    openai_api_key = arquivo.read().strip()
print("Chave da OpenAI carregada com sucesso.")

openai.api_key = openai_api_key

EMBEDDING_MODEL = "text-embedding-ada-002"

def get_embedding(text):
    """Generate an embedding for the given text using OpenAI's API."""

    # Check for valid input
    if not text or not isinstance(text, str):
        return None

    try:
        # Call OpenAI API to get the embedding
        embedding = openai.embeddings.create(input=text, model=EMBEDDING_MODEL).data[0].embedding
        return embedding
    except Exception as e:
        print(f"Error in get_embedding: {e}")
        return None

dataset_df["plot_embedding_optimised"] = dataset_df['Resumo'].apply(get_embedding)


dataset_df.head()

Chave da OpenAI carregada com sucesso.


,Tipo do documento,Nº do documento,Ano de emissão,Área,Laboratorio,CRD,Cliente,CNPJ/CPF/CRD do cliente,Interessado,Titulo,Nº do projeto,Projeto de capacitação,Cancelado e substituido por,Formato do documento,Data de emissão,Descritores,Resumo,Gerente,Equipe,plot_embedding_optimised
0,Relatório Técnico,166547,2023,TRM,LV,DH14,GASCAT INDUSTRIA E COMERCIO LTDA,00965449000194,NaN,AVALIAÇÃO DE UM SISTEMA DE MEDIÇÃO DE VOLUME T...,810103C,810103C,NaN,Digital,24/03/2023 18:36:16,ROTATIVO;TEMPERATURA;PRESSÃO,Avaliação de um sistema de medição de volume t...,RUBENS SILVA TELLES,Autor: Jose Eduardo Prata;Equipe: Fernando Hen...,"[-0.005251199938356876, 0.017263857647776604, ..."
1,Relatório Técnico,166548,2023,TRM,LV,DH14,GASCAT INDUSTRIA E COMERCIO LTDA,00965449000194,NaN,AVALIAÇÃO DE UM SISTEMA DE MEDIÇÃO DE VOLUME T...,810103C,810103C,NaN,Digital,24/03/2023 18:37:29,ROTATIVO;TEMPERATURA;PRESSÃO,Avaliação de um sistema de medição de volume t...,RUBENS SILVA TELLES,Autor: Jose Eduardo Prata;Equipe: Fernando Hen...,"[-0.005251199938356876, 0.017263857647776604, ..."
2,Relatório Técnico,166549,2023,TRM,LV,DH14,GASCAT INDUSTRIA E COMERCIO LTDA,00965449000194,NaN,AVALIAÇÃO DE UM SISTEMA DE MEDIÇÃO DE VOLUME T...,810103C,810103C,NaN,Digital,24/03/2023 18:38:37,TURBINA;TEMPERATURA;PRESSÃO,Avaliação de um sistema de medição de volume t...,RUBENS SILVA TELLES,Autor: Jose Eduardo Prata;Equipe: Fernando Hen...,"[-0.005251199938356876, 0.017263857647776604, ..."
3,Relatório Técnico,166550,2023,TRM,LV,DH14,GASCAT INDUSTRIA E COMERCIO LTDA,00965449000194,NaN,AVALIAÇÃO DE UM SISTEMA DE MEDIÇÃO DE VOLUME T...,810103C,810103C,NaN,Digital,17/04/2023 17:26:32,TURBINA;TEMPERATURA;PRESSÃO,Avaliação de um sistema de medição de volume t...,RUBENS SILVA TELLES,Autor: Jose Eduardo Prata;Equipe: Fernando Hen...,"[-0.005251199938356876, 0.017263857647776604, ..."
4,Relatório Técnico,166672,2023,MA,LCPE,DO13,FUNDO NACIONAL DE DESENVOLVIMENTO DA EDUCACAO,00378257000181,NaN,ANÁLISE DA QUALIDADE FÍSICA DE LIVROS ESCOLARE...,500209B,não,NaN,Digital,22/09/2023 12:51:03,LIVRO ESCOLAR; FNDE; PNLD.,Este Relatório Técnico apresenta os resultados...,PATRÍCIA KAJI YASUMURA,Autor: PATRÍCIA KAJI YASUMURA;Equipe: PATRICIA...,"[-0.0022202394902706146, 0.009880401194095612,..."


In [ ]:
# Convertendo cada número em cada embedding para float porque o querido mongodb quer double para fazer indice vetorial
dataset_df["plot_embedding_optimised"] = dataset_df["plot_embedding_optimised"].apply(
    lambda embeddings: [float(num) for num in embeddings]
)

# Verificando o resultado após a conversão
print(dataset_df["plot_embedding_optimised"].head())


0    [-0.005251199938356876, 0.017263857647776604, ...
1    [-0.005251199938356876, 0.017263857647776604, ...
2    [-0.005251199938356876, 0.017263857647776604, ...
3    [-0.005251199938356876, 0.017263857647776604, ...
4    [-0.0022202394902706146, 0.009880401194095612,...
Name: plot_embedding_optimised, dtype: object


## Step 5: Vector Database Setup and Data Ingestion

MongoDB acts as both an operational and a vector database. It offers a database solution that efficiently stores, queries and retrieves vector embeddings—the advantages of this lie in the simplicity of database maintenance, management and cost.

**CRIE UM USUÁRIO SOMENTE COM LETRAS E NUMEROS**

**To create a new MongoDB database, set up a database cluster:**

1. Head over to MongoDB official site and register for a [free MongoDB Atlas account](https://www.mongodb.com/cloud/atlas/register), or for existing users, [sign into MongoDB Atlas](https://account.mongodb.com/account/login?nds=true).

2. Select the 'Database' option on the left-hand pane, which will navigate to the Database Deployment page, where there is a deployment specification of any existing cluster. Create a new database cluster by clicking on the "+Create" button.

3.   Select all the applicable configurations for the database cluster. Once all the configuration options are selected, click the “Create Cluster” button to deploy the newly created cluster. MongoDB also enables the creation of free clusters on the “Shared Tab”.

 *Note: Don’t forget to whitelist the IP for the Python host or 0.0.0.0/0 for any IP when creating proof of concepts.*

4. After successfully creating and deploying the cluster, the cluster becomes accessible on the ‘Database Deployment’ page.

5. Click on the “Connect” button of the cluster to view the option to set up a connection to the cluster via various language drivers.

6. This tutorial only requires the cluster's URI(unique resource identifier). Grab the URI and copy it into the Google Colabs Secrets environment in a variable named `MONGO_URI` or place it in a .env file or equivalent.










In [ ]:
#Isso aqui voce deve pegar do mongo e colar aqui, substituir por seu usuário e senha
from pymongo.mongo_client import MongoClient
uri = "mongodb+srv://aa:aa@cluster0.5plsrjn.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0"#usar senha e usuário só com letras e números
# Create a new client and connect to the server
client = MongoClient(uri)
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [ ]:
import pymongo

def get_mongo_client(mongo_uri):
    """Estabelece conexão com o MongoDB."""
    try:
        client = pymongo.MongoClient(mongo_uri)
        print("Conexão com o MongoDB bem-sucedida")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Falha na conexão: {e}")
        return None

# Substitua '<password>' pela sua senha real
mongo_uri = uri

mongo_client = get_mongo_client(mongo_uri)

# Ingestão de dados no MongoDB
db = mongo_client['resumos']
collection = db['resumos_relatorios']



Conexão com o MongoDB bem-sucedida


In [ ]:
# Delete any existing records in the collection
collection.delete_many({})

DeleteResult({'n': 2494, 'electionId': ObjectId('7fffffff0000000000000004'), 'opTime': {'ts': Timestamp(1710859102, 1632), 't': 4}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1710859102, 1635), 'signature': {'hash': b'\x0bL\xe8\xc5\x16\xf4\x97\xfbr\xf0\x18\x04c\xe9\x97U\x05\xfeR\x88', 'keyId': 7340831160269799430}}, 'operationTime': Timestamp(1710859102, 1632)}, acknowledged=True)

In [ ]:
documents = dataset_df.to_dict('records')
collection.insert_many(documents)

print("Data ingestion into MongoDB completed")

Data ingestion into MongoDB completed


*texto em itálico*## Step 6: Create a Vector Search Index

Essa parte é chata, clicar no nome do cluster, vai aparecer um menu grande, clique em atlas search, create search index, atlas vector search e next.Coloque o nome do indice (lembre de colocar nome igual no código), escolha a coleção, preencha direitinho os campos abaixo:

{
  "fields": [
    {
      "type": "vector",
      "path": "<field-name>", #nome de onde estão os embeddings
      "numDimensions": 0,#numero de dimensões dos embeddings
      "similarity": "<similarity-function>" Qual função vai usar, coseno, etc.
    }
  ]
}

Eu já copiei no código o jeito certo de fazer a query que tem relação com essa parte. Tem um exemplo de query certa dentro do mongodb. Se fizer diferente não funciona.

{
  "fields": [
    {
      "numDimensions": 1536,
      "path": "plot_embedding_optimised",
      "similarity": "euclidean",
      "type": "vector"
    }
  ]
}

Esse é como ficou o conteúdo do meu indice no mongo.

At this point make sure that your vector index is created via MongoDB Atlas.
Follow instructions here:

This next step is mandatory for conducting efficient and accurate vector-based searches based on the vector embeddings stored within the documents in the ‘movie_collection’ collection. Creating a Vector Search Index enables the ability to traverse the documents efficiently to retrieve documents with embeddings that match the query embedding based on vector similarity. Go here to read more about [MongoDB Vector Search Index](https://www.mongodb.com/docs/atlas/atlas-search/field-types/knn-vector/).


## Step 7: Perform Vector Search on User Queries

This step combines all the activities in the previous step to provide the functionality of conducting vector search on stored records based on embedded user queries.

This step implements a function that returns a vector search result by generating a query embedding and defining a MongoDB aggregation pipeline. The pipeline, consisting of the `$vectorSearch` and `$project` stages, queries using the generated vector and formats the results to include only required information like plot, title, and genres while incorporating a search score for each result.

This selective projection enhances query performance by reducing data transfer and optimizes the use of network and memory resources, which is especially critical when handling large datasets. For AI Engineers and Developers considering data security at an early stage, the chances of sensitive data leaked to the client side can be minimized by carefully excluding fields irrelevant to the user's query.


In [ ]:
def vector_search(query, collection):
    # Obtemos o vetor de consulta a partir da função get_embedding.
    query_embedding = get_embedding(query)

    # Verificar se conseguimos obter um embedding válido para a consulta.
    if query_embedding is None:
        print("Não foi possível obter o embedding para a consulta.")
        return []

    # Pipeline de agregação usando o estágio $vectorSearch conforme as diretrizes do mongo, em algum lugar no site tem isso
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_indexdri",  # Substitua pelo nome real do seu índice vetorial configurado no MongoDB Atlas
                "path": "plot_embedding_optimised",  # O campo que contém os vetores de embeddings nos documentos
                "queryVector": query_embedding,  # O vetor de consulta gerado pela função get_embedding
                "numCandidates": 150,  # Número de candidatos a considerar durante a busca vetorial
                "limit": 5,  # Limita o número de resultados retornados pela busca
                # Opcionalmente, você pode adicionar um filtro aqui, se necessário, não entendi o que é esse filtro, ler no site do mongo
                # "filter": {<especificação-do-filtro>}
            }
        },
        {
            "$project": {  # Projeção para especificar quais campos devem ser incluídos no resultado final, coloquei só esses pra testar, os mesmos tem que por no outro código abaixo
                "Titulo": 1,
                "Nº do documento": 1,
                "Ano de emissão": 1,
                "score": {"$meta": "searchScore"}  # Inclui a pontuação de similaridade da busca vetorial
            }
        }
    ]

    # Executamos a busca usando o pipeline de agregação.
    try:
        results = list(collection.aggregate(pipeline))
        return results
    except Exception as e:
        print(f"Erro ao realizar a busca vetorial: {str(e)}")
        return []




## Step 8: Handling User Query and Result

The final step in the implementation phase focuses on the practical application of our vector search functionality and AI integration to handle user queries effectively.

The handle_user_query function performs a vector search on the MongoDB collection based on the user's query and utilizes OpenAI's GPT-3.5 model to generate context-aware responses.


In [ ]:
def handle_user_query(query, collection):

  get_knowledge = vector_search(query, collection)

  search_result = ''
  for result in get_knowledge:#colocar os nomes das coisas que quer que apareçam na busca e estão na tabela de relatórios
      search_result += f"Titulo: {result.get('Titulo', 'N/A')}, Nº do documento: {result.get('Nº do documento', 'N/A')}, Ano de emissão: {result.get('Ano de emissão', 'N/A')}\n"

  completion = openai.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
          {"role": "system", "content": "Você é um sistema de buscas de relatórios do Instituto de Pesquisas Tecnológicas do IPT."},
          {"role": "user", "content": "Responda a query do usuário em português: " + query + " with the following context: " + search_result}
      ]
  )

  return (completion.choices[0].message.content), search_result



In [ ]:
# 6. Conduct query with retrival of sources
query = "Ache um relatório sobre barulho"
response, source_information = handle_user_query(query, collection)

print(f"Response: {response}")
print(f"Source Information: \n{source_information}")

Response: O relatório que corresponde à sua busca é o seguinte:  
- Título: ELABORAÇÃO DE DIRETRIZES PARA MITIGAÇÃO DE RUÍDO GERADO PELA ARENA ALLIANZ PARQUE, PERCEBIDO NAS RESIDÊNCIAS NO ENTORNO DA ARENA  
- Nº do documento: 169546  
- Ano de emissão: 2023
Source Information: 
Titulo: ELABORAÇÃO DE DIRETRIZES PARA MITIGAÇÃO DE RUÍDO GERADO PELA ARENA ALLIANZ PARQUE, PERCEBIDO NAS RESIDÊNCIAS NO ENTORNO DA ARENA, Nº do documento: 169546, Ano de emissão: 2023
Titulo: ENSAIOS DE CAMPO PARA A AVALIAÇÃO DE VAGÕES GDU, Nº do documento: 170592, Ano de emissão: 2023
Titulo: MEDIDAS COM O PÊNDULO BRITÂNICO NO PAVIMENTO DE VIAS INTERNAS DO HOTEL ROSEWOOD SÃO PAULO, Nº do documento: 170132, Ano de emissão: 2023
Titulo: PROGRAMA INTERLABORATORIAL PARA ENSAIOS EM CHAPAS DE PAPELÃO ONDULADO CICLO 2023 – 1ª RODADA, Nº do documento: 169292, Ano de emissão: 2023
Titulo: LEVANTAMENTO DAS ATUAIS CONDIÇÕES ESTRUTURAIS,RELACIONADAS AO DESENVOLVIMENTO DE PATOLOGIAS, DE 6 IMÓVEIS LOCALIZADOS NO ENTORNO DAS 